In [1]:
#MAIN SUMO IMPLEMENTATION FOR OUR PROJECT
import traci
import pandas as pd
from os import path

# Connect to SUMO simulation
traci.start(["sumo", "-c", path.abspath("test.sumocfg")])

# Function definitions
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def getNextBusInRoute(vehicleId, distances):
    if (len(distances) == 0 or len(distances) == 1):  return ""
    sortedDists = sorted(list(distances.values()))
    nextDistanceValueInRoute = sortedDists[(sortedDists.index(distances[vehicleId]) + 1) % len(sortedDists)]
    return [{key: values} for key, values in distances.items() if values == nextDistanceValueInRoute]

def calculateDistanceBusStopAndOtherBusStop():
    # finds bus stop on route
    position = traci.vehicle.getLanePosition(vehicleId)
    edgeId = traci.vehicle.getRoadID(vehicleId)

    busLocation = 0.0
    otherEdgeId = ""

    # handles case if the edge is a junction
    # otherwise, we simply assign the other route
    if edgeId not in busLocationDict:
        parsedJunction = edgeId.split("_")[0].replace(":", "")
        adjacentEdges = intersection(junctionDict[parsedJunction], route_names[route_index])

        # creates a list of tuples, each containing an edge and its index
        indexed_edges = [(edge, route_names[route_index].index(edge)) for edge in adjacentEdges]

        # sorts the list by index
        indexed_edges.sort(key=lambda x: x[1])

        # gets the edges with the smallest and largest index
        edgeId, _ = indexed_edges[0]
        otherEdgeId, _ = indexed_edges[-1]

        # when the route repeats at the last road, the last road should be the previous lane, and the first road should be the next lane
        # thus not following the pattern of smallest and largest index
        if(edgeId == "-overlap" and otherEdgeId == "-L0" or 
            edgeId == "-overlap" and otherEdgeId == "-R0"):
            other = -1
            position = traci.lane.getLength(otherEdgeId + "_0")
        else:
            # otherwise, the direction is as expected, and we set it to 1.
            other = 1
            position = traci.lane.getLength(edgeId + "_0")
        busLocation = float(busLocationDict[edgeId])
    else:
        busLocation = float(busLocationDict[edgeId])
        other = 1 if position > busLocation else -1     # 1 for other as the next lane, -1 for other as the previous lane
        otherEdgeId = route_names[route_index][(route_names[route_index].index(edgeId) + other) % len(route_names[route_index])]
        
    routeLength = traci.lane.getLength(edgeId + "_0")

    ## finds other bus stop on either previous or next route
    otherBusLocation = float(busLocationDict[otherEdgeId])
    otherRouteLength = traci.lane.getLength(otherEdgeId + "_0")

    distanceBusLocation = round(abs(position - busLocation), 3)
    distanceOtherBusLocation = round(position + otherRouteLength - otherBusLocation if other == -1 else routeLength - position + otherBusLocation, 3)

    return distanceBusLocation, distanceOtherBusLocation, other


# initializations
df3 = pd.DataFrame(
    {
    "Step": [],
    "Id": [],
    "Route": [],
    "NextBusStop": [],
    "DistanceToNextBus": [],
    "PedestrianCount": [],
    "AverageWaitTime": [],
    }
)

repeats = 100
route0_edges = traci.route.getEdges("r_0")
route1_edges = traci.route.getEdges("r_1")

route_lengths = [sum(traci.lane.getLength(edge + "_0") for edge in route0_edges) / repeats, sum(traci.lane.getLength(edge + "_0") for edge in route1_edges) / repeats]
route_busstops = [["bs_road_-overlap", "bs_road_-R2", "bs_road_-R1", "bs_road_-R0"], ["bs_road_-overlap", "bs_road_-L3", "bs_road_-L2", "bs_road_-L1", "bs_road_-L0"]]

busLocationDict={"-overlap": "123", "-R2": "259", "-R1": "125", "-R0": "267", "-L3": "117", "-L2": "110", "-L1": "123", "-L0": "120"}
junctionDict={"J1": ["-L0", "-R0", "-overlap"], "J2": ["-R0", "-R1"], "J3": ["-R1", "-R2"], "J4": ["-L3", "-R2", "-overlap"], "J5": ["-L0", "-L1"], "J7": ["-L1", "-L2"], "J8": ["-L2", "-L3"]}
route_names = [["-overlap", "-R2", "-R1", "-R0"], ["-overlap", "-L3", "-L2", "-L1", "-L0"]]

# simulation loop
step = 0
while step < 500:
    traci.simulationStep()

    vehicles = traci.vehicle.getIDList()
    persons = traci.person.getIDList()
 
    step += 1

    distanceDrivenInRoutesList = [{}, {}] # contains both routes in different lists with the busses positions on the routes
    personsWaitingTimeList = []

    df3StartLength = len(df3)

    for i in range(0, len(vehicles)):
        vehicleId = vehicles[i]
        vehicleRoute = traci.vehicle.getRouteID(vehicleId)
        route_index = 0 if vehicleRoute == "r_0" else 1  # r_0 = route_index=0, r_1 = route_index=1

        # inserts the step and route of the vehicle into the dataframe
        df3.loc[len(df3)] = {"Step": step, "Id": vehicleId, "Route": vehicleRoute, "PedestrianCount": len(persons)}

        ### calculate distance driven on the route and insert it into the dfRoutes list
        vehicleDistanceDriven = round(traci.vehicle.getDistance(vehicleId) % (route_lengths[0] if (vehicleRoute == "r_0") else route_lengths[1]), 3)
        distanceDrivenInRoutesList[route_index][vehicleId] = vehicleDistanceDriven

        ### now calculate the distance to the previous and next bus stop
        distanceBusLocation, distanceOtherBusLocation, other = calculateDistanceBusStopAndOtherBusStop()
        
        df3.loc[df3StartLength + i, "NextBusStop"] = distanceBusLocation if other == -1 else distanceOtherBusLocation
    
    # finds person waiting time
    for j in range(0,len(persons)):
        personWaitingTime = traci.person.getWaitingTime(persons[j])
        personsWaitingTimeList.append(personWaitingTime)

    # inserts the distance to the next bus stop and the average waiting time into the dataframe
    for i in range(0, len(vehicles)):
        busDistances = distanceDrivenInRoutesList[0] if traci.vehicle.getRouteID(vehicles[i]) == "r_0" else distanceDrivenInRoutesList[1]
        vehicleId = vehicles[i]
        length = len(personsWaitingTimeList)
        averageWaitTime = sum(personsWaitingTimeList)/length if length > 0 else 0

        df3.loc[df3StartLength + i, "DistanceToNextBus"] = str(getNextBusInRoute(vehicleId, busDistances))
        df3.loc[df3StartLength + i, "AverageWaitTime"] = averageWaitTime

print(df3)
df3.to_csv("output.csv", index=False)
traci.close() 

C:\Users\sebas\AppData\Local\Temp\ipykernel_20380\1065616026.py:142: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df3.loc[df3StartLength + i, "DistanceToNextBus"] = str(getNextBusInRoute(vehicleId, busDistances))


   Step         Id Route  NextBusStop DistanceToNextBus  PedestrianCount  \
0     1  bus_r_0_0   r_0        115.9                                  2   

   AverageWaitTime  
0              1.0  


In [6]:
## OTHER IMPLEMENTATION WITH ADDITIONAL INFORMATION, JUST LET IT BE HERE FOR NOW
import traci
import pandas as pd
from os import path


# Connect to SUMO simulation
traci.start(["sumo", "-c", path.abspath("test.sumocfg")])

df = pd.DataFrame(
    {
    "Step": [],
    "Id": [],
    "Speed": [],
    "Capacity": [],
    "Route": "",
    "DistanceDriven": []
    })

df2 = pd.DataFrame(
    {
    "Step": [],
    "Id": [],
    "WaitingTime": []
    })

repeats = 100
route0_edges = traci.route.getEdges("r_0")
route1_edges = traci.route.getEdges("r_1")

route_lengths = [sum(traci.lane.getLength(edge + "_0") for edge in route0_edges) / repeats, sum(traci.lane.getLength(edge + "_0") for edge in route1_edges) / repeats]
route_busstops = [["bs_road_-overlap", "bs_road_-R2", "bs_road_-R1", "bs_road_-R0"], ["bs_road_-overlap", "bs_road_-L3", "bs_road_-L2", "bs_road_-L1", "bs_road_-L0"]]

step = 0
while step < 500:
    traci.simulationStep()

    vehicles = traci.vehicle.getIDList()
    persons = traci.person.getIDList()

    step += 1
    
    #https://sumo.dlr.de/docs/TraCI/Vehicle_Value_Retrieval.html
    #https://sumo.dlr.de/pydoc/traci._vehicle.html#VehicleDomain-getSpeed
    distanceDrivenInRoutesList = [[], []]

    dfStartLength = len(df)
    for i in range(0,len(vehicles)):
        vehicleId = vehicles[i]
        vehicleSpeed = round(traci.vehicle.getSpeed(vehicleId)*3.6, 2) # m/s to km/h
        vehicleCapacity = traci.vehicle.getPersonCapacity(vehicleId)
        vehicleWaitingTime = traci.vehicle.getWaitingTime(vehicleId)
        vehicleRoute = traci.vehicle.getRouteID(vehicleId)
        vehicleDistanceDriven = round(traci.vehicle.getDistance(vehicleId) % (route_lengths[0] if (vehicleRoute == "r_0") else route_lengths[1]), 3)
        if(vehicleRoute == "r_0"):
            distanceDrivenInRoutesList[0].append((vehicleId,vehicleDistanceDriven))
        else:
            distanceDrivenInRoutesList[1].append((vehicleId,vehicleDistanceDriven))
        df.loc[len(df)] = {"Step": step, "Id": vehicleId, "Speed": vehicleSpeed, "Capacity": vehicleCapacity, "Route": vehicleRoute, "DistanceDriven": vehicleDistanceDriven}

    for i in range(0, len(vehicles)):
        df.loc[dfStartLength + i, "DistanceDriven"] = str(distanceDrivenInRoutesList[0] if traci.vehicle.getRouteID(vehicles[i]) == "r_0" else distanceDrivenInRoutesList[1])
    
    # https://sumo.dlr.de/pydoc/traci._person.html
    for j in range(0,len(persons)):
        personId = persons[j]
        personWaitingTime = traci.person.getWaitingTime(personId)
        df2.loc[len(df2)] = {"Step": step, "Id": personId, "WaitingTime": personWaitingTime}

    # ML code/funcs here
    # control vehicles 

print()
print("---- VEHICLES ----")
print(df)

print("---- PERSONS ----")
print(df2)
        
traci.close() 


C:\Users\sebas\AppData\Local\Temp\ipykernel_25536\3318629610.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[('bus_r_0_0', 0.0)]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[dfStartLength + i, "DistanceDriven"] = str(distanceDrivenInRoutesList[0] if traci.vehicle.getRouteID(vehicles[i]) == "r_0" else distanceDrivenInRoutesList[1])
